In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
AbbreviationDictionary = {"BankNote": "BN",
                          "Bar7": "B7",
                          "BreastCancer": "BC",
                          "CarEvaluation": "CE",
                          "COMPAS": "CP",
                          "FICO": "FI",
                          "Haberman": "HM",
                          "Iris": "IS",
                          "MONK1": "M1",
                          "MONK3":"M3"}

In [3]:
BatchSize = 5
DiversityWeight = 0.4

In [4]:
# Input Data Set #
Data = "BankNote" # Iris  MONK1  MONK3  Bar7 (10)  COMPAS (50) | BankNote (5)  BreastCancer (5)  CarEvaluation (5)  FICO (50)  Haberman 
JobNameAbbrev = AbbreviationDictionary[Data]

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                      #  "Seed":list([]),                       
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["BatchQBCDiversityFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.045],
                       "Type": ["Classification"],
                       "DiversityWeight": [DiversityWeight],
                       "BatchSize": [BatchSize],
                       "Partition": ["medium"],                                                        # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                       "Time": ["2-23:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": ["30000M"]}                                                                # [100M, 30000M, 100000M]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Passive Learning

In [5]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["PassiveLearning"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "DiversityWeight": [0],
                       "BatchSize": [BatchSize],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Include/exclude Random Forest QBC Simulations

In [6]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["BatchQBCDiversityFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "DiversityWeight": [DiversityWeight],
                       "BatchSize": [BatchSize],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [7]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str)+
    "_D" + ParameterVector["DiversityWeight"].astype(str) + 
    "B" + ParameterVector["BatchSize"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI1_", "_PL", regex=True)
    .str.replace(r"0.", "", regex=False)
    )

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

---

# Only run these ones

In [8]:
# ### Simulations that failed ###
FilterText = ['2BN_DA045_D4B5', '6BN_DA045_D4B5', '8BN_DA045_D4B5', '18BN_DA045_D4B5', '21BN_DA045_D4B5', '22BN_DA045_D4B5', '25BN_DA045_D4B5', '39BN_DA045_D4B5', '2BN_UA045_D4B5', '6BN_UA045_D4B5', '7BN_UA045_D4B5', '8BN_UA045_D4B5', '13BN_UA045_D4B5', '21BN_UA045_D4B5', '31BN_UA045_D4B5', '33BN_UA045_D4B5', '38BN_UA045_D4B5', '39BN_UA045_D4B5']

### Rerun these simulations ###
ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
ParameterVector.index = range(0,len(ParameterVector))
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,DiversityWeight,BatchSize,Partition,Time,Memory,JobName,Output
0,BankNote,2,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,2BN_DA045_D4B5,BankNote/TreeFarms/Raw/2BN_DA045_D4B5.pkl
1,BankNote,2,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,2BN_UA045_D4B5,BankNote/TreeFarms/Raw/2BN_UA045_D4B5.pkl
2,BankNote,6,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,6BN_UA045_D4B5,BankNote/TreeFarms/Raw/6BN_UA045_D4B5.pkl
3,BankNote,6,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,6BN_DA045_D4B5,BankNote/TreeFarms/Raw/6BN_DA045_D4B5.pkl
4,BankNote,7,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,7BN_UA045_D4B5,BankNote/TreeFarms/Raw/7BN_UA045_D4B5.pkl
5,BankNote,8,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,8BN_UA045_D4B5,BankNote/TreeFarms/Raw/8BN_UA045_D4B5.pkl
6,BankNote,8,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,8BN_DA045_D4B5,BankNote/TreeFarms/Raw/8BN_DA045_D4B5.pkl
7,BankNote,13,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,13BN_UA045_D4B5,BankNote/TreeFarms/Raw/13BN_UA045_D4B5.pkl
8,BankNote,18,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,18BN_DA045_D4B5,BankNote/TreeFarms/Raw/18BN_DA045_D4B5.pkl
9,BankNote,21,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,5,medium,2-23:59:00,30000M,21BN_DA045_D4B5,BankNote/TreeFarms/Raw/21BN_DA045_D4B5.pkl


---

# Save Parameter Vector

In [9]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)